[<img align="left" width="140px" src="https://storage.googleapis.com/static.drivetrainhub.com/img/dh_logo_text_217x80.png"/>](https://drivetrainhub.com)

<div align="right">© 2019 Drivetrain Hub LLC</div>

# Strength / Gear Tooth Contact Stress
---

**Authors**:  [Tugan Eritenel](https://github.com/tugan1) 

**Description**:  Calculation of gear tooth contact stress in spur gears.

## Table of Contents

1. [Introduction](#Introduction)
2. [Contact Stress Theory](#Contact-Stress-Theory)
3. [Gear Contact Stress Calculation](#Gear-Contact-Stress-Calculation)
    1. [Example](#Example-1)
4. [Conclusion](#Conclusion)
5. [References](#References)

## Introduction

This chapter is on contact stress of parallel axis gears.  

#### Notebook imports and settings

In [1]:
import ipywidgets as widgets
from IPython import display
from IPython.core.display import HTML
import pandas as pd
import math

## Contact Stress Theory

Gear tooth contact along the tooth surface can be considered as a line. The width of the contact is much shorter than the length along the line of contact.  The radius of curvatures of the contacting surfaces vary as the gears rotate.  Maximum subsurface shear stress is just below the point of contact.  The mechanism of stress is quite complicated due to sliding and friction between the surfaces.  

<img src="./img/contact.png" alt="Involute parameters" style="width: 100%; max-width: 650px"/>
<p style="text-align: center; font-weight: bold;">Stresses on gear tooth.</p>
<span style="font-size: 8pt; float: right">Image credit: G. M. Maitra, Handbook of Gear Design </span>
<br>

Pitting is a type of gear failure as a result of subsurface contact stress cycles.  The contact stress is denoted with the symbol $p$ referring to Hertzian pressure developed.  The maximum pressure on two cylinders in contact is given by,

$$ p_{c(max)} = \frac{4 F}{\pi B L}$$ 

where the width of the contact is calculated as,

$$B = \sqrt{\frac{8 F}{\pi L} \frac{\left(1-\nu_1^2\right)/E_1 + \left(1-\nu_2^2\right)/E_2}{\left(1/D_1\right) + \left(1/D_2\right)}}$$

## Gear Contact Stress Calculation

When the elastic modulus of the two cylinders are the same $E_1 = E_2$, and poisson ratio are $\nu_1 = \nu_2 = 0.3$, the contact pressure for gears becomes,

$$ p_p = \sqrt{0.35\frac{F_t E}{b d_{p,1}} \frac{i+1}{i} \frac{1}{\cos^2\alpha \tan\alpha_w}} $$

A few parameters are factored so that from the equation above as follows,

$$ p_p = y_m y_p \sqrt{\frac{F_t}{b_1 d_{p,1}} \frac{i+1}{i}}$$

where material factor $y_m$ and tooth shape factor, $y_p$, are given by, 

$$y_m = \sqrt{0.35 E}$$ 

$$y_p = \sqrt{\frac{1}{\cos^2\alpha \tan\alpha_w}} $$ 

For a first design iteration, assuming no profile shift, and $\alpha = 20^{\circ}$, $y_p = 1.76$, and $E=206,000$ MPa, so $y_m = 269$.

Maximum allowable surface fatigue stress $p_{sc}$ needs to be determined from the gear material.  For example, induction hardened 40 Cr 4 allowable surface fatigue stress is 1620 MPa.  Generally a factor of safety of 1.5 is applied, giving the contact pressure limit as $p_{cp}< \frac{p_{sc}}{1.5}$.

Contact pressure is written in terms of applied torque as,

$$ p_{cp} = y_m y_p\sqrt{ \frac{2000 T_1}{\left(b/d_{p,1}\right) d_{p,1}^3} \frac{i+1}{i}}$$

Minimum module to satisfy contact pressure safety is found according to,

$$ m_n > \frac{1}{z_1} \sqrt[3]{ \frac{2000 T_1 y_m^2 y_p^2}{\left(b/d_{p,1}\right) p_{cp}^2} \frac{i+1}{i}}$$

<div class="alert alert-block alert-warning" style="box-shadow: 2px 2px 4px 0px rgba(0, 0, 0, .13);">
    <h3>Example | Tooth Contact Stress<a name="Example-1"></a></h3>
    <h4>GIVEN</h4>
    - A gear pair has 45 teeth on the pinion and 137 teeth on the wheel.<br>
    - Torque on the pinion is 600 Nm.<br>
    <h4>FIND</h4>
    1) Minimum module so that the pinion is safe for contact fatigue.<br>
    2) Contact pressure on pinion tooth flank.
    <h4>SOLUTION</h4>
    For the first iteration of design $y_p = 1.76$ and $y_m=269$.  

The ratio of facewidth to pitch diameter is taken as $b_1 / d_{p,1} = 0.5$ to provide a starting point for the design. The formula below is used for finding the minimum module. Substituting the given values, the minimum module for contact is:
</div>

In [2]:
T_1 = 600
z_1 = 45
z_2 = 137
i=z_2/z_1
y_m = 269
y_p = 1.76
facewidth_to_pitch = 0.5
p_cp = 1620/1.5


m = 1/z_1*math.pow(2000*T_1 * (y_m**2) * (y_p**2)/((facewidth_to_pitch)*p_cp**2)*(i+1)/i ,1/3)
display.display(HTML('<div class="alert alert-block alert-info"> Minimum module for bending stress, $m$ = ' + str(round(m,2)) + '</div>'))

m_nearest = math.ceil(m*2)/2
display.display(HTML('<div class="alert alert-block alert-info"> Selected module, $m_{n}$ = ' + str(round(m_nearest,2)) + '</div>'))


<div class="alert alert-block alert-warning" style="box-shadow: 2px 2px 4px 0px rgba(0, 0, 0, .13);">
    For the selected module, the contact pressure is calculated as:
</div>

In [4]:
m = m_nearest
d_p1 = m * z_1
F_t = 2000 * T_1 / d_p1
p_cp = y_m * y_p * math.sqrt(2000 * T_1 / (facewidth_to_pitch * d_p1 ** 3) * (i + 1) / i)

display.display(HTML('<div class="alert alert-block alert-info"> Contact pressure on pinion, $p_{cp}$ = ' + str(round(p_cp,2)) + ' MPa </div>'))

## Conclusion

This notebook documents the procedure to find spur gear contact stress.  Inputs are a few parameters from gear macro geometry: number of teeth, pressure angle, and applied torque.  Minimum module is found so that the gears are safe considering contact stress.  For a chosen module and the gear parameters above, contact stress is calculated.

---

<div class="alert alert-block alert-info" style="box-shadow: 2px 2px 4px 0px rgba(0, 0, 0, .13);">
    <h3>Learn More</h3><br>
    Continue reading the <a href="https://drivetrainhub.com/notebooks/">Drivetrain Hub | Notebook Series</a> to learn more about the design and analysis of spur gears and other gear types.
</div>

<div class="alert alert-block alert-info" style="box-shadow: 2px 2px 4px 0px rgba(0, 0, 0, .13);">
    <h3>Model Gear Contact</h3><br>
    Accurately model the contact of a gear mesh with the <a href="https://drivetrainhub.com/gears">Drivetrain Hub | Gears App</a>, a modern drivetrain modeling environment 100% online at <a href="https://drivetrainhub.com">drivetrainhub.com</a>.
</div>

<div class="alert alert-block alert-info" style="box-shadow: 2px 2px 4px 0px rgba(0, 0, 0, .13);">
    <h3>Improve Notebook</h3><br>
    Our gear geometry notebooks are publicly hosted in a <a href="https://github.com/drivetrainhub/notebooks-gears/">GitHub repository</a>, available for anyone to view and propose edits.
</div>

# References

1. G. M. Maitra, (1994), "Handbook of Gear Design", Tata McGraw-Hill